In [ ]:
# for speech-to-text
import speech_recognition as sr
# for text-to-speech
import pyttsx3
import os
import time
# for data
import datetime
import numpy as np
import torch
# import model class and tokenizer
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
# import pywhatkit
# import wikipedia

In [ ]:
# download and setup model and tokenizer
tokenizer = BlenderbotTokenizer.from_pretrained('C:/data/blenderbot-3B/blenderb',map_location=torch.device('cuda'))
model = BlenderbotForConditionalGeneration.from_pretrained('C:/data/blenderbot-3B/blenderb')

In [ ]:
# Building the AI
class ChatBot():
    def __init__(self, name):
        print("----- Starting up", name, "-----")
        self.name = name
    def listen(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as mic:
            print("Listening...")
            audio = recognizer.listen(mic)
            self.text="ERROR"
        try:
            self.text = recognizer.recognize_google(audio)
            print("Me  --> ", self.text)
        except:
            print("Me  -->  ERROR")
    @staticmethod
    def talk(text):
        print("Dev --> ", text)
        engine = pyttsx3.init('sapi5')
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[3].id)
        engine.setProperty('rate',200)
        engine.say(text)
        engine.runAndWait()
    def wake_up(self, text):
        return True if self.name in text.lower() else False
    @staticmethod
    def action_time():
        return datetime.datetime.now().time().strftime('%H:%M')

In [ ]:
# Running the AI
if __name__ == "__main__":
    ai = ChatBot(name="bob")
    #nlp = transformers.pipeline("conversational", model="microsoft/DialoGPT-medium")
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    ex=True
    while ex:
        ai.listen()
        ## wake up
        if ai.wake_up(ai.text) is True:
            chat = model.generate(**tokenizer(ai.text, return_tensors='pt'))
            res = tokenizer.decode(chat[0])
            res = res.replace("<s>", "")
            res = res.replace("</s>", "")
            
        ## action time
        elif "time" in ai.text:
            res = ai.action_time()
        elif any(i in ai.text for i in ["exit","close"]):
            res = np.random.choice(["Tata","Have a good day","Bye","Goodbye","Hope to meet soon","peace out!"])
            ex=False        
        else:   
            if ai.text=="ERROR":
                res=""
            else:
                chat = model.generate(**tokenizer(ai.text, return_tensors='pt'))
                res = tokenizer.decode(chat[0])
                res = res.replace("<s>", "")
                res = res.replace("</s>", "")
        ai.talk(res)
    print("----- Closing down Bob -----")